# Create master df
Total observations = 50,000
<br>0 = 25,000, 15,000 from sentiment 140, 10,000 from amazon reviews
<br>1 = 25,000, 10,000 from sentiment 140, 5000 from consumer complaints, 10000 from amazon reviews
<br> All observations from sentiment 140_neutlabels

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', 0) #To display entire text content of a column
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
from sklearn.model_selection import train_test_split
import os
from functools import reduce
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords 

In [3]:
os.chdir('D:\capstone_data\processed_csv\duplicates_rm')

In [12]:
#Read all dfs
sentiment_140_df = pd.read_csv('sentiment140_no_dplcts_textcol.csv')
sentiment_140_neutlabels =  pd.read_csv('sentiment_140_neutlabels_no_dplcts_textcol.csv')
consumer_complaints_df = pd.read_csv('consumer_complaints_no_dplcts_textcol.csv')
amazonreviews_df = pd.read_csv('amazonreviews_with_label_no_dplcts_textcol.csv')

In [5]:
#Extract 15,000 0s from sentiment 140
sentiment_140_0s_15k = sentiment_140_df[sentiment_140_df['label']==0].sample(
    n = 15000, replace = False, random_state = 7)
sentiment_140_0s_15k['label'].value_counts()

0    15000
Name: label, dtype: int64

In [6]:
#Extract 10,000 0s from amazon reviews
amazonreviews_0s_10k = amazonreviews_df[amazonreviews_df['label'] == 0].sample(
n = 10000, replace = False, random_state = 7)
amazonreviews_0s_10k['label'].value_counts()

0    10000
Name: label, dtype: int64

In [7]:
#Extract all 0s from sentiment_140_neutlabels
sentiment_140_neutlabels_0s = sentiment_140_neutlabels[sentiment_140_neutlabels['label'] == 0]
sentiment_140_neutlabels_0s['label'].value_counts()

0    321
Name: label, dtype: int64

In [8]:
#Concat all 0s in master df 
master_df = pd.concat([sentiment_140_0s_15k, amazonreviews_0s_10k, sentiment_140_neutlabels_0s],
                     axis = 0, ignore_index = True)

In [9]:
#Verify
display(master_df.tail(2))
display(master_df['label'].value_counts())

,text,label
25319,Ask Programming: LaTeX or InDesign?: submitted by calcio1 [link] [1 comment] http://tinyurl.com/myfmf7,0
25320,Ahhh... back in a *real* text editing environment. I &lt;3 LaTeX.,0


0    25321
Name: label, dtype: int64

In [7]:
#Create a function to extract data points from df
def extract_data_pts(df, label, n_pts):
    df = df[df['label'] == label].sample(
    n = n_pts, replace = False, random_state = 7)
    return df

In [11]:
#Extract 10,000 1s from sentiment 140
sentiment_140_1s_10k = extract_data_pts(sentiment_140_df,1,10000)
display(sentiment_140_1s_10k['label'].value_counts())
display(sentiment_140_1s_10k.head(2))

1    10000
Name: label, dtype: int64

,text,label
548091,MISSING MY BOYFRIEND,1
163918,@SquishT77 So sorry to hear about your mum.,1


In [12]:
#Extract 5,000 1s from consumer complaints
consumer_complaints_1s_5k = extract_data_pts(consumer_complaints_df,1,5000)
consumer_complaints_1s_5k['label'].value_counts()

1    5000
Name: label, dtype: int64

In [13]:
#Extract 10,000 1s from amazon
amazon_reviews_1s_10k = extract_data_pts(amazonreviews_df,1,10000)
amazon_reviews_1s_10k['label'].value_counts()

1    10000
Name: label, dtype: int64

In [14]:
#Extract all 1s from amazon_reviews_1s_10k
sentiment_140_neutlabels_1s = sentiment_140_neutlabels[sentiment_140_neutlabels['label'] == 1]
sentiment_140_neutlabels_1s['label'].value_counts()

1    177
Name: label, dtype: int64

In [15]:
#Concat all 4 dfs above and append to master_df
master_df = master_df.append(
    pd.concat([sentiment_140_1s_10k,consumer_complaints_1s_5k,
    amazon_reviews_1s_10k, sentiment_140_neutlabels_1s],
                          axis = 0, ignore_index = True)
    , ignore_index = True)

In [16]:
#Verify master_df
display(master_df.shape)
display(master_df.tail(2))
display(master_df['label'].value_counts())

(50498, 2)

,text,label
50496,"Trouble in Iran, I see. Hmm. Iran. Iran so far away. #flockofseagullsweregeopoliticallycorrect",1
50497,Reading the tweets coming out of Iran... The whole thing is terrifying and incredibly sad...,1


0    25321
1    25177
Name: label, dtype: int64

In [18]:
#Write master_df to csv
master_df.to_csv('master_df_snmnt_&cmplnts.csv', index = False)

# Data Cleaning 

In [4]:
#Read 
master_df = pd.read_csv('master_df_snmnt_&cmplnts.csv')
master_df.head()

,text,label
0,@vallywally LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?,0
1,@k80bug Ohhh I used to make those when I was operations there,0
2,That is...if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops.,0
3,"Good stuff med @ericries Strax ï¿½ver en timme lï¿½ng, check it! http://bit.ly/15Vumv Tack @emileifrem !!",0
4,Tomorrow I begin a two day shoot for an online retailer. I'll be posing in various garments... I love it!,0


## Data cleaning steps
1. Remove user handles - e.g. @broskiii OH SNAP YOU WORK......
2. Remove hashtags  - e.g. #at&amp;t is complete fail
3. Remove urls.
4. Adress contractions e.g. 'didn't' by expanding contractions via contracion map. The expanded contracion for 'didn't will be 'did not. ***(Added later, in 2nd iteration)*** 
5. Replace punctuations with whitespace - For intial model we will focus on key words to determine negative tweets.
6. Remove numbers
7. Trim repeated letters in a word - People tend to use mutiple letters to emphasize especially on twitter e.g. I am soooooooooo hungryyyyyyyyyyyyyyyyyy. Any letter repeating more than twice will be trimmed. Hence, the mentioned text will be modified to I am soo hungryy.
<br>
<br>It must be noted that it is very important to complete steps 1 through 3 before replacing punctuations with whitespace because they all include some form of punctuation like @,#,// etc. that will be removed and alphanumeric values are left unchanged. e.g. "http://twitpic.com/5ej4y - My favorite neurotic sister" will be transformed to "http   twitpic com 5ej4y   My favorite neurotic sister"






In [31]:
#Smaller sample from master to test our functions
df = master_df.head(25)
df

,text,label
0,@vallywally LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?,0
1,@k80bug Ohhh I used to make those when I was operations there,0
2,That is...if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops.,0
3,"Good stuff med @ericries Strax ï¿½ver en timme lï¿½ng, check it! http://bit.ly/15Vumv Tack @emileifrem !!",0
4,Tomorrow I begin a two day shoot for an online retailer. I'll be posing in various garments... I love it!,0
5,@hillkath Good morning.,0
6,"In Melbourne Australia, Thursday 7th May there is a Body Balance info night, tasting and free bottle! Direct message me email for details",0
7,@therealswizzz http://twitpic.com/5eyah - XOXO,0
8,@SashaKane have a happy day,0
9,@VivaLaCaro That was coolio! &lt;3 I left reviews and fatty fuel.,0


In [32]:
# Step 1. Remove user handles
df['clean_text'] = df['text'].apply(lambda x: re.sub("@[\w]*","", x))
#@ = literal match, @[] = @ followed by, \w = Aplhanumeric , * = 0 or more occurences
#Verify
df.head(2)

C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,text,label,clean_text
0,@vallywally LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?,0,LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?
1,@k80bug Ohhh I used to make those when I was operations there,0,Ohhh I used to make those when I was operations there


In [33]:
# Step 2. Remove hashtags
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub("#[\w]*","", x))
#Verify
df.tail(2)

C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,text,label,clean_text
23,@mileycyrus hi miley i loved you're movie it was amazing !,0,hi miley i loved you're movie it was amazing !
24,still keep crocheting n thinking if i should watch #fringe as well #crochet,0,still keep crocheting n thinking if i should watch as well


In [34]:
#Step 3. Remove urls
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub("http\S+", "", x))
# http = literal match, \S -all non white spaces, + = one or more occurences
#Verify
df.head(10)

C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,text,label,clean_text
0,@vallywally LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?,0,LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?
1,@k80bug Ohhh I used to make those when I was operations there,0,Ohhh I used to make those when I was operations there
2,That is...if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops.,0,That is...if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops.
3,"Good stuff med @ericries Strax ï¿½ver en timme lï¿½ng, check it! http://bit.ly/15Vumv Tack @emileifrem !!",0,"Good stuff med Strax ï¿½ver en timme lï¿½ng, check it! Tack !!"
4,Tomorrow I begin a two day shoot for an online retailer. I'll be posing in various garments... I love it!,0,Tomorrow I begin a two day shoot for an online retailer. I'll be posing in various garments... I love it!
5,@hillkath Good morning.,0,Good morning.
6,"In Melbourne Australia, Thursday 7th May there is a Body Balance info night, tasting and free bottle! Direct message me email for details",0,"In Melbourne Australia, Thursday 7th May there is a Body Balance info night, tasting and free bottle! Direct message me email for details"
7,@therealswizzz http://twitpic.com/5eyah - XOXO,0,- XOXO
8,@SashaKane have a happy day,0,have a happy day
9,@VivaLaCaro That was coolio! &lt;3 I left reviews and fatty fuel.,0,That was coolio! &lt;3 I left reviews and fatty fuel.


In [35]:
###########Step 4. Expand contractions
#Define contracion map
c_dict = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "cos": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "I would",
    "I'd": "I would",
    "i'd've": "I would have",
    "I'd've": "I would have",
    "i'll": "I will",
    "I'll": "I will",
    "i'll've": "I will have",
    "I'll've": "I will have",
    "i'm": "I am",
    "I'm": "I am",
    "i've": "I have",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'alls": "you alls",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had",
    "you'd've": "you would have",
    "you'll": "you you will",
    "you'll've": "you you will have",
    "you're": "you are",
    "you've": "you have"
}

In [36]:
# Compiling the contraction dict
c_re = re.compile('(%s)' % '|'.join(c_dict.keys()))

#Function for expanding contracions
def expand_contractions(sentence, c_re = c_re):
    def replace(match):
        return c_dict[match.group(0)]
    return c_re.sub(replace, sentence)

In [37]:
#Verify
df['clean_text'] = df['clean_text'].apply(lambda x: expand_contractions(x))
df.head(10)
#Index 4 - I'll converted to I will

C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,text,label,clean_text
0,@vallywally LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?,0,LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?
1,@k80bug Ohhh I used to make those when I was operations there,0,Ohhh I used to make those when I was operations there
2,That is...if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops.,0,That is...if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops.
3,"Good stuff med @ericries Strax ï¿½ver en timme lï¿½ng, check it! http://bit.ly/15Vumv Tack @emileifrem !!",0,"Good stuff med Strax ï¿½ver en timme lï¿½ng, check it! Tack !!"
4,Tomorrow I begin a two day shoot for an online retailer. I'll be posing in various garments... I love it!,0,Tomorrow I begin a two day shoot for an online retailer. I will be posing in various garments... I love it!
5,@hillkath Good morning.,0,Good morning.
6,"In Melbourne Australia, Thursday 7th May there is a Body Balance info night, tasting and free bottle! Direct message me email for details",0,"In Melbourne Australia, Thursday 7th May there is a Body Balance info night, tasting and free bottle! Direct message me email for details"
7,@therealswizzz http://twitpic.com/5eyah - XOXO,0,- XOXO
8,@SashaKane have a happy day,0,have a happy day
9,@VivaLaCaro That was coolio! &lt;3 I left reviews and fatty fuel.,0,That was coolio! &lt;3 I left reviews and fatty fuel.


In [38]:
#Step 4. Replace punctuations except for ! with whitespace
#Get punctuations
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [39]:
#Replace punctuation with whitespace
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub("""['!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~']""",
                                                                  " ", x))
df.head(10)

C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,text,label,clean_text
0,@vallywally LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?,0,LOL Damm U make commision when SHE wins NICEE BTW What do u play when ure there
1,@k80bug Ohhh I used to make those when I was operations there,0,Ohhh I used to make those when I was operations there
2,That is...if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops.,0,That is if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops
3,"Good stuff med @ericries Strax ï¿½ver en timme lï¿½ng, check it! http://bit.ly/15Vumv Tack @emileifrem !!",0,Good stuff med Strax ï¿½ver en timme lï¿½ng check it Tack
4,Tomorrow I begin a two day shoot for an online retailer. I'll be posing in various garments... I love it!,0,Tomorrow I begin a two day shoot for an online retailer I will be posing in various garments I love it
5,@hillkath Good morning.,0,Good morning
6,"In Melbourne Australia, Thursday 7th May there is a Body Balance info night, tasting and free bottle! Direct message me email for details",0,In Melbourne Australia Thursday 7th May there is a Body Balance info night tasting and free bottle Direct message me email for details
7,@therealswizzz http://twitpic.com/5eyah - XOXO,0,XOXO
8,@SashaKane have a happy day,0,have a happy day
9,@VivaLaCaro That was coolio! &lt;3 I left reviews and fatty fuel.,0,That was coolio lt 3 I left reviews and fatty fuel


In [40]:
#Remove numbers, code below will remove all non alphabets and convert to lowercase
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub('[^A-Za-z]', ' ', x.lower()))
#^ = Negation
#Verify
df.head(10)

C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,text,label,clean_text
0,@vallywally LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?,0,lol damm u make commision when she wins nicee btw what do u play when ure there
1,@k80bug Ohhh I used to make those when I was operations there,0,ohhh i used to make those when i was operations there
2,That is...if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops.,0,that is if i can make it thru the west and southside of montgomery without being stopped by the punkass cops
3,"Good stuff med @ericries Strax ï¿½ver en timme lï¿½ng, check it! http://bit.ly/15Vumv Tack @emileifrem !!",0,good stuff med strax ver en timme l ng check it tack
4,Tomorrow I begin a two day shoot for an online retailer. I'll be posing in various garments... I love it!,0,tomorrow i begin a two day shoot for an online retailer i will be posing in various garments i love it
5,@hillkath Good morning.,0,good morning
6,"In Melbourne Australia, Thursday 7th May there is a Body Balance info night, tasting and free bottle! Direct message me email for details",0,in melbourne australia thursday th may there is a body balance info night tasting and free bottle direct message me email for details
7,@therealswizzz http://twitpic.com/5eyah - XOXO,0,xoxo
8,@SashaKane have a happy day,0,have a happy day
9,@VivaLaCaro That was coolio! &lt;3 I left reviews and fatty fuel.,0,that was coolio lt i left reviews and fatty fuel


In [41]:
###############Remove letters repeated more than twice in succession
#Create a function
def trim_repeated_letters(sentence):
    return(reduce(lambda x,y: x+y if x[-2:] != y*2 else x, sentence, ""))
    

#Apply function to sample_df
df['clean_text'] = df['clean_text'].apply(lambda x: trim_repeated_letters(x) )
df.head()

C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,text,label,clean_text
0,@vallywally LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?,0,lol damm u make commision when she wins nicee btw what do u play when ure there
1,@k80bug Ohhh I used to make those when I was operations there,0,ohh i used to make those when i was operations there
2,That is...if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops.,0,that is if i can make it thru the west and southside of montgomery without being stopped by the punkass cops
3,"Good stuff med @ericries Strax ï¿½ver en timme lï¿½ng, check it! http://bit.ly/15Vumv Tack @emileifrem !!",0,good stuff med strax ver en timme l ng check it tack
4,Tomorrow I begin a two day shoot for an online retailer. I'll be posing in various garments... I love it!,0,tomorrow i begin a two day shoot for an online retailer i will be posing in various garments i love it


All data cleaning steps used above will be compiled into one **df cleaning function** as it will have to be used multiple times

In [45]:
#contarction map
c_dict = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "cos": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "I would",
    "I'd": "I would",
    "i'd've": "I would have",
    "I'd've": "I would have",
    "i'll": "I will",
    "I'll": "I will",
    "i'll've": "I will have",
    "I'll've": "I will have",
    "i'm": "I am",
    "I'm": "I am",
    "i've": "I have",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'alls": "you alls",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had",
    "you'd've": "you would have",
    "you'll": "you you will",
    "you'll've": "you you will have",
    "you're": "you are",
    "you've": "you have"
}
#Contraction
c_re = re.compile('(%s)' % '|'.join(c_dict.keys()))


In [46]:
###################################Functions
#Function for expanding contracions
def expand_contractions(sentence, c_re = c_re):
    def replace(match):
        return c_dict[match.group(0)]
    return c_re.sub(replace, sentence)

#Repeated letters func
def trim_repeated_letters(sentence):
    return(reduce(lambda x,y: x+y if x[-2:] != y*2 else x, sentence, ""))

#Compile allsteps above into one function for future use
def text_df_cleaner(df):
    '''
    Cleans 'text' column of a df and creates a new column 'clean_text' with clean text
    '''
    df['clean_text'] = df['text'].apply(lambda x: re.sub("@[\w]*","", x)) #User handles
    df['clean_text'] = df['clean_text'].apply(lambda x: re.sub("#[\w]*","", x)) #hash tags
    df['clean_text'] = df['clean_text'].apply(lambda x: re.sub("http\S+", "", x)) #urls
    df['clean_text'] = df['clean_text'].apply(lambda x: expand_contractions(x)) #expand contractions
    df['clean_text'] = df['clean_text'].apply(lambda x: re.sub("""['!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~']""",
                                                                  " ", x)) #Punctuations
    df['clean_text'] = df['clean_text'].apply(lambda x: re.sub('[^A-Za-z]', ' ', x.lower())) #Numbers and lowercase
    #Repeated letters
    df['clean_text'] = df['clean_text'].apply(lambda x: trim_repeated_letters(x))
    return df
    

In [47]:
#Verify cleaning function
trial_df = master_df.tail(20)
text_df_cleaner(trial_df)
trial_df.head()
#Function is working properly

C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

,text,label,clean_text
50478,"HATE safeway select green tea icecream! bought two cartons, what a waste of money. &gt;_&lt;",1,hate safeway select green tea icecream bought two cartons what a waste of money gt lt
50479,argghhhh why won't my jquery appear in safari bad safari !!!,1,argghh why will not my jquery appear in safari bad safari
50480,"I can't watch TV without a Tivo. And after all these years, the Time/Warner DVR STILL sucks. http://www.davehitt.com/march03/twdvr.html",1,i cannot watch tv without a tivo and after all these years the time warner dvr still sucks
50481,I still love my Kindle2 but reading The New York Times on it does not feel natural. I miss the Bloomingdale ads.,1,i still love my kindle but reading the new york times on it does not feel natural i miss the bloomingdale ads
50482,"Although today's keynote rocked, for every great announcement, AT&amp;T shit on us just a little bit more.",1,although today s keynote rocked for every great announcement at amp t shit on us just a little bit more


In [50]:
#Clean master_df and write to csv for modelling
text_df_cleaner(master_df)

,text,label,clean_text
0,@vallywally LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?,0,lol damm u make commision when she wins nicee btw what do u play when ure there
1,@k80bug Ohhh I used to make those when I was operations there,0,ohh i used to make those when i was operations there
2,That is...if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops.,0,that is if i can make it thru the west and southside of montgomery without being stopped by the punkass cops
3,"Good stuff med @ericries Strax ï¿½ver en timme lï¿½ng, check it! http://bit.ly/15Vumv Tack @emileifrem !!",0,good stuff med strax ver en timme l ng check it tack
4,Tomorrow I begin a two day shoot for an online retailer. I'll be posing in various garments... I love it!,0,tomorrow i begin a two day shoot for an online retailer i will be posing in various garments i love it
5,@hillkath Good morning.,0,good morning
6,"In Melbourne Australia, Thursday 7th May there is a Body Balance info night, tasting and free bottle! Direct message me email for details",0,in melbourne australia thursday th may there is a body balance info night tasting and free bottle direct message me email for details
7,@therealswizzz http://twitpic.com/5eyah - XOXO,0,xoxo
8,@SashaKane have a happy day,0,have a happy day
9,@VivaLaCaro That was coolio! &lt;3 I left reviews and fatty fuel.,0,that was coolio lt i left reviews and fatty fuel


In [51]:
#Verify
master_df.head(10000).tail(10)

,text,label,clean_text
9990,Just got back from my first ever Fun Run. And it definitely won't be the last.,0,just got back from my first ever fun run and it definitely will not be the last
9991,@KimSherrell hope you have a wonderful weekend,0,hope you have a wonderful weekend
9992,@jenniferdodd i do show some discretion occasionally,0,i do show some discretion occasionally
9993,@PumaSwede your a chelsea fan aww im not ur friend now lol,0,your a chelsea fan aww im not ur friend now lol
9994,@musicaddikt787 hey crys and I think iloveyoutoo,0,hey crys and i think iloveyoutoo
9995,"@kickyoutube hey, can i ask.... why cant i use AVI when i download? and kickyoutube is very usefule... it's nice!",0,hey can i ask why cant i use avi when i download and kickyoutube is very usefule it is nice
9996,In french! Tis rather amusing watching Mr Brian trying to control the class! x,0,in french tis rather amusing watching mr brian trying to control the class x
9997,@murdles You better believe it! I would never miss our guys in my own city! Are you?,0,you better believe it i would never miss our guys in my own city are you
9998,Visiting @emdeee,0,visiting
9999,"@unkleEL hmm, I would say the same thing about ppl turning into zombies at I'm a Mac commercials-but 2 each its own",0,hmm i would say the same thing about ppl turning into zombies at i am a mac commercials but each its own


In [52]:
#Drop text column and write to csv
master_df.drop(columns =['text'], inplace = True)
master_df.head(20000).tail(2)

,label,clean_text
19998,0,after a while of looking for the combination of specifications that owns this team stumbled upon this so far has been excellent purchase not esconere that the battery heats up a little more than usual but with a cooling fan we have solved this inconvenience
19999,0,awsome awsome awsome phone does everything great price great value samsung has hit the mark with this one note


In [53]:
#Write to csv
master_df.to_csv('D:\capstone_data\processed_csv\master_data_clean_sent140_cmplnts_mixed\sent140_cmplnts_50k_clean.csv',
                index = False)

In [54]:
#Verify csv
df = pd.read_csv('D:\capstone_data\processed_csv\master_data_clean_sent140_cmplnts_mixed\sent140_cmplnts_50k_clean.csv')
display(df.head())
display(df.shape)

,label,clean_text
0,0,lol damm u make commision when she wins nicee btw what do u play when ure there
1,0,ohh i used to make those when i was operations there
2,0,that is if i can make it thru the west and southside of montgomery without being stopped by the punkass cops
3,0,good stuff med strax ver en timme l ng check it tack
4,0,tomorrow i begin a two day shoot for an online retailer i will be posing in various garments i love it


(50498, 2)